In [2]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

# Generate synthetic training and validation data
train_data = torch.randn((1000, 3, 224, 224))  # Example: 1000 training samples
train_labels = torch.randint(0, 2, (1000,))  # Example: Binary classification labels
val_data = torch.randn((200, 3, 224, 224))  # Example: 200 validation samples
val_labels = torch.randint(0, 2, (200,))  # Example: Binary classification labels

# Define data preprocessing and augmentation transforms
preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the synthetic dataset
class SyntheticDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        x = self.data[index]
        y = self.labels[index]
        
        if self.transform:
            x = self.transform(x)
        
        return x, y

train_dataset = SyntheticDataset(train_data, train_labels, transform=preprocess)
val_dataset = SyntheticDataset(val_data, val_labels, transform=preprocess)

# Define data loaders
batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

base_model = models.resnet50(pretrained=True)
num_features = base_model.fc.in_features

# Replace the fully connected layer for object detection
num_classes = 2  # Example: 2 classes (binary classification)
base_model.fc = nn.Linear(num_features, num_classes)

# Transfer the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = base_model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    model.train()
    for inputs, labels in train_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in val_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        accuracy = 100 * correct / total
        print('Epoch {}: Loss {:.4f} | Validation Accuracy {:.2f}%'.format(epoch+1, running_loss / len(train_dataloader), accuracy))


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 275MB/s]


Epoch 1: Loss 0.7342 | Validation Accuracy 58.00%
Epoch 2: Loss 0.3150 | Validation Accuracy 51.00%
Epoch 3: Loss 0.0794 | Validation Accuracy 50.50%
Epoch 4: Loss 0.0803 | Validation Accuracy 49.00%
Epoch 5: Loss 0.1839 | Validation Accuracy 48.50%
Epoch 6: Loss 0.4670 | Validation Accuracy 55.00%
Epoch 7: Loss 0.3651 | Validation Accuracy 44.50%
Epoch 8: Loss 0.2052 | Validation Accuracy 43.50%
Epoch 9: Loss 0.1203 | Validation Accuracy 48.50%
Epoch 10: Loss 0.0494 | Validation Accuracy 49.00%
